In [1]:
# Supress Warnings

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Import the numpy and pandas packages

import numpy as np
import pandas as pd

In [3]:
## Checkpoints - Part 1
#Checkpoint 1: Data Cleaning 1
#Load the companies and rounds data (provided on the previous page) into two data frames 
#and name them companies and rounds2 respectively.

companies = pd.read_csv("../bhassrin/companies.txt", delimiter="\t", encoding='iso-8859-1')
#companies
companies.shape
rounds2 = pd.read_csv("../bhassrin/rounds2.csv", encoding='iso-8859-1')

#round
rounds2.shape
companies
rounds2

# Cleaning the special characters in company_permalink column of rounds2 DataFrame

# Check whether these particular field has special characters or not as mentioned in assignment.
# similarly check for few other columns using integer index using iloc functionality.

rounds2['company_permalink'].iloc[29597]
rounds2['company_permalink'].iloc[29597]
rounds2['company_permalink'].iloc[31863]
rounds2['company_permalink'].iloc[45176]
rounds2['company_permalink'].iloc[58473]
rounds2['company_permalink'].iloc[101036]
rounds2['company_permalink'].iloc[109969]
rounds2['company_permalink'].iloc[113839]

#Now remove the special characters for rounds2 and comapanies DataFrames.

rounds2['company_permalink'] = rounds2['company_permalink'].apply(lambda x: x.encode('utf-8').decode('ascii', 'ignore'))

companies['permalink'] = companies['permalink'].apply(lambda x: x.encode('utf-8').decode('ascii', 'ignore'))
companies['name'] = companies['name'].apply(lambda x: str(x).encode('utf-8').decode('ascii', 'ignore'))

#Check whether the special character is removed or not using integer indexes again.

#Now convert unique ids present in both the dataframes to lower case.

rounds2.company_permalink = rounds2.company_permalink.astype(str).str.lower()
companies.permalink = companies.permalink.astype(str).str.lower()


rounds2 = rounds2.rename(columns={'company_permalink': 'permalink'})


In [4]:

#Table 1.1: Understand the Data Set 

#How many unique companies are present in rounds2?

#len(rounds2['company_permalink'].tolist()) #114949
#len(companies['permalink'].tolist()) #66368
#
len(rounds2['permalink'].unique().tolist())
#How many unique companies are present in the companies file?
len(companies['permalink'].unique().tolist())
#Are there any companies in the rounds2 file which are not  present in companies ? Answer Y/N.
rnd = set(rounds2['permalink'])
rnd
comp = set(companies['permalink'])
comp
len(rnd.intersection(comp))
len(rnd.difference(comp))

0

In [6]:
companies.shape
rounds2.shape

(66368, 10)

In [7]:
#Data cleanup rounds2

rounds2.head(100)
rounds2[rounds2.isnull().sum(axis=1) > 5]
len(rounds2[rounds2.isnull().sum(axis=1) > 5].index)

round(100*(rounds2.isnull().sum()/len(rounds2.index)), 2)
rounds2 = rounds2[~np.isnan(rounds2['raised_amount_usd'])]
round(100*(rounds2.isnull().sum()/len(rounds2.index)), 2)
#rounds2 = rounds2.drop(['funding_round_code'],   axis=1)
round(100*(rounds2.isnull().sum()/len(rounds2.index)), 2)
rounds2.shape

(94959, 6)

In [8]:
#Data cleanup Companies
#Identify rows with more than 5 missing values
companies[companies.isnull().sum(axis=1) > 5]
len(companies[companies.isnull().sum(axis=1) > 5].index)


round(100*(companies.isnull().sum()/len(companies.index)), 2)
#companies = companies.drop([ 'state_code', 'region', 'city','founded_at','homepage_url'],   axis=1)

round(100*(companies.isnull().sum()/len(companies.index)), 2)

# Similarly  for category_list and countr_code the missing values are still there. Identify them and drop.
# First replace them with NaN and then drop using dropna.
#companies.shape

companies['category_list'].replace('', np.nan, inplace=True)
companies['country_code'].replace('', np.nan, inplace=True)
companies.dropna(subset=['category_list'], inplace=True)
companies.dropna(subset=['country_code'], inplace=True)
round(100*(companies.isnull().sum()/len(companies.index)), 2)
companies.shape
rounds2.shape

(94959, 6)

In [10]:
#Checkpoint 2: Funding Type Analysis
#Table-1.1 Understand the Data Set 
#Merge the two data frames so that all  variables (columns)  in the companies frame are added to the rounds2 data frame. 
#Name the merged frame master_frame. 
#Merging two dataframes based on shared index values 


#companies.columns # permalink

master_frame = pd.merge(companies, rounds2, on='permalink', how = 'left')

rounds2.columns #company_permalink
companies.columns
#master_frame
master_frame.columns
master_frame.shape
#master_frame.info()

len(master_frame['permalink'].unique().tolist())


57804

In [11]:
#Average Values of Investments Table 2.1.1
master_frame.shape
master_frame
#Now we have to do group analysis,
# Grouping: First, we will group the dataframe by Funding Types i.e., 'funding_round_type'

invest_sub_type = ['venture', 'angel', 'seed', 'private_equity']
invest_sub_type_df = master_frame.loc[master_frame['funding_round_type'].isin(invest_sub_type), :]
invest_sub_type_df['raised_amount_usd'] = invest_sub_type_df['raised_amount_usd']/1000000
invest_sub_type_grp = invest_sub_type_df.groupby('funding_round_type')
invest_sub_type_grp['raised_amount_usd'].mean().sort_values(ascending = False)



funding_round_type
private_equity    73.938486
venture           11.724223
angel              0.971574
seed               0.747794
Name: raised_amount_usd, dtype: float64

In [12]:
#Checkpoint 3: Country Analysis
#Analysing the Top 3 English-Speaking Countries
#country analysis.
#Top nine countries which have received the highest total funding (across ALL sectors for the chosen investment type)

master_frame

#Replace this chosen_invest_type with the funding type as required. Here in example taken 'venture' as chosen type.
invest_sub_type = ['venture', 'angel', 'seed', 'private_equity']


#Prepare a dataframe for all values of chosen investment type.
chosen_invest_type_df = master_frame.loc[master_frame['funding_round_type'].isin(invest_sub_type), :]
chosen_invest_type_df

#Now set the index to prepare a new dataframe top9.
df = chosen_invest_type_df.set_index(['country_code','raised_amount_usd'])    
top_funding_types_for_chosen = chosen_invest_type_df.groupby(['country_code'])['raised_amount_usd'].sum().reset_index()

#Now make a data frame named top9 with the top nine countries based on the total investment amount each country has received.
top9 = top_funding_types_for_chosen.nlargest(9, 'raised_amount_usd')
top9


,country_code,raised_amount_usd
124,USA,5.084739e+11
21,CHN,6.100092e+10
40,GBR,2.689981e+10
53,IND,2.104911e+10
18,CAN,1.316891e+10
29,DEU,8.580103e+09
57,ISR,8.101823e+09
39,FRA,8.078614e+09
88,NLD,5.058484e+09


In [13]:
top9_csv = top9.to_csv (r'C:\Users\bhassrin\top9.csv', index = None, header=True)

In [15]:
#Checkpoint 4: Sector Analysis 1

#Sector Analysis for country_code ['USA', 'GBR', 'IND'] and funding_round_type in ['venture']
chosen_countries = ['USA', 'GBR', 'IND'] 
venture_df_1 = master_frame.loc[master_frame['funding_round_type'].isin(['venture']), :]

venture_df = venture_df_1.loc[venture_df_1['country_code'].isin(chosen_countries), :]


#1.Extract the primary sector of each category list from the category_list column
venture_df['primary_sector'] = venture_df.category_list.str.split("|", n = 2, expand = True)[0]

#2 Load the mapping.csv file into a new dataframe mappings
mappings = pd.read_csv("C:\\Users\\BHASSRIN\mapping.csv")
#Drop unnecessary columns and clean for NaN
mappings = mappings.drop(['Blanks'],   axis=1)
mappings.columns
mappings.dropna(subset=['category_list'], inplace=True)
mappings

#Now use melt function to transform the mappings dataframe as required.

melted = pd.melt(mappings, id_vars=["category_list"], var_name="Main_Sector", value_name="isMain")
melted.shape

# Filter only those rows which are mapped to main sectors.
mappings_final = melted[(melted['isMain'] == '1')]

#Drop unnecessary columns
mappings_final = mappings_final.drop(['isMain'],   axis=1)
mappings_final

#Create a final dataframe with primary_sector and main_sector columns. Merge both to get the final dataframe.
venture_df_final = pd.merge(venture_df, mappings_final, on='category_list')
venture_df_final.columns

#Now this final Dataframe is mapped to one of 8 main sectors. Be happy now.
len(venture_df_final['Main_Sector'].unique().tolist())

8

In [111]:
#Check point 5
#1. Create three separate data frames D1, D2 and D3 for each of the three countries containing the observations 
# of funding type FT falling within the 5-15 million USD range. The three data frames should contain:

#All the columns of the master_frame along with the primary sector and the main sector
#The total number (or count) of investments for each main sector in a separate column
#The total amount invested in each main sector in a separate column

venture_df_5_to_15M = venture_df_final.loc[(round(venture_df_final.raised_amount_usd/1000000,0) > 5.0) & (round(venture_df_final.raised_amount_usd/1000000,0) < 15.0) , :]

#The below Data Frame will contain all the investments ranging 5M to 15M for Venture funding type and Top 3 English speaking countries.

venture_df_5_to_15M['country_code'].unique().tolist()

master_frame.columns
Country_1 = ['USA']
Country_2 = ['GBR']
Country_3 = ['IND']


#Prepare a dataframe for all values of chosen country_code.
D1_temp = venture_df_5_to_15M.loc[venture_df_5_to_15M['country_code'].isin(Country_1), :]
D2_temp = venture_df_5_to_15M.loc[venture_df_5_to_15M['country_code'].isin(Country_2), :]
D3_temp = venture_df_5_to_15M.loc[venture_df_5_to_15M['country_code'].isin(Country_3), :]

#Now set the index to prepare a new dataframes D1, D2, D3
#For this first prepare dataframes with columns as temporary data frames and then merge them with main data frame
#using .Main_Sector

df1 = D1_temp.set_index(['Main_Sector','raised_amount_usd'])    
D1_temp_amount = D1_temp.groupby(['Main_Sector'])['raised_amount_usd'].sum().reset_index()
D1_temp_amount = D1_temp_amount.rename(columns={'raised_amount_usd': 'total_amount_raised'})
df11 = D1_temp.set_index(['Main_Sector','raised_amount_usd'])    
D1_temp_count = D1_temp.groupby(['Main_Sector'])['raised_amount_usd'].size().reset_index()
D1_temp_count = D1_temp_count.rename(columns={'raised_amount_usd': 'total_investments'})


df2 = D2_temp.set_index(['Main_Sector','raised_amount_usd'])    
D2_temp_amount = D2_temp.groupby(['Main_Sector'])['raised_amount_usd'].sum().reset_index()
D2_temp_amount = D2_temp_amount.rename(columns={'raised_amount_usd': 'total_amount_raised'})
df22 = D2_temp.set_index(['Main_Sector','raised_amount_usd'])    
D2_temp_count = D2_temp.groupby(['Main_Sector'])['raised_amount_usd'].size().reset_index()
D2_temp_count = D2_temp_count.rename(columns={'raised_amount_usd': 'total_investments'})


df3 = D3_temp.set_index(['Main_Sector','raised_amount_usd'])    
D3_temp_amount = D3_temp.groupby(['Main_Sector'])['raised_amount_usd'].sum().reset_index()
D3_temp_amount = D3_temp_amount.rename(columns={'raised_amount_usd': 'total_amount_raised'})
df33 = D3_temp.set_index(['Main_Sector','raised_amount_usd'])    
D3_temp_count = D3_temp.groupby(['Main_Sector'])['raised_amount_usd'].size().reset_index()
D3_temp_count = D3_temp_count.rename(columns={'raised_amount_usd': 'total_investments'})

D1_amt_cnt = pd.merge(D1_temp_amount, D1_temp_count, on='Main_Sector')
D2_amt_cnt = pd.merge(D2_temp_amount, D2_temp_count, on='Main_Sector')
D3_amt_cnt = pd.merge(D3_temp_amount, D3_temp_count, on='Main_Sector')


D1 = pd.merge(D1_temp, D1_amt_cnt, on='Main_Sector')
D2 = pd.merge(D2_temp, D2_amt_cnt, on='Main_Sector')
D3 = pd.merge(D3_temp, D3_amt_cnt, on='Main_Sector')

#Table 5.1

#1 Total number of Investments (count)
D1['total_investments'].unique().sum()
D2['total_investments'].unique().sum()
D3['total_investments'].unique().sum()

#2 Total amount of investment (USD)
D1['total_amount_raised'].unique().sum()
D2['total_amount_raised'].unique().sum()
D3['total_amount_raised'].unique().sum()

#Top Sector name (no. of investment-wise) For questions 3 to 8
D1_temp_count.sort_values(by=['total_investments'])
D2_temp_count.sort_values(by=['total_investments'])
D3_temp_count.sort_values(by=['total_investments'])



#For point 3 (top sector count-wise), which company received the highest investment?
# top sector is Others for D1 
OTH = ['Others']
D1_others = D1.loc[D1['Main_Sector'].isin(OTH), :]
D1_others.sort_values(by=['raised_amount_usd'], ascending =False)
D1_others.nlargest(1, 'raised_amount_usd')
#For point 4 (second best sector count-wise), which company received the highest investment?
#For Table - 5.1 Questions 9 and 10 in table,

CLN = ['Cleantech / Semiconductors']
D1_CLN = D1.loc[D1['Main_Sector'].isin(CLN), :]
#D1_CLN.sort_values(by=['raised_amount_usd'], ascending =False).nlargest('Main_Sector',1)
D1_CLN.nlargest(1, 'raised_amount_usd')


#Same to be repeated for D2
D2_others = D2.loc[D2['Main_Sector'].isin(OTH), :]
D2_others.sort_values(by=['raised_amount_usd'], ascending =False)

D2_CLN = D2.loc[D2['Main_Sector'].isin(CLN), :]
D2_CLN.sort_values(by=['raised_amount_usd'], ascending =False)

#Same to be repeated for D3

NSM = ['News, Search and Messaging']

#D3_others = D3.loc[D3['Main_Sector'].isin(OTH), :]
#D3_others.sort_values(by=['raised_amount_usd'], ascending =False)

D3_NSM = D3.loc[D3['Main_Sector'].isin(NSM), :]
D3_NSM.sort_values(by=['raised_amount_usd'], ascending =False)






In [122]:
#Checkpoint 6 
#Below code will export the data to a csv file from master_frame data frame
master_frame = master_frame.to_csv (r'C:\Users\bhassrin\master_frame.csv', index = None, header=True) 